# Collective Endowment Effects ABM

In [4]:
import random
import warnings

import polars as pl
import altair as alt
from mesa import Agent
from mesa import Model
from mesa.time import RandomActivation
from mesa.space import NetworkGrid
import networkx as nx

warnings.filterwarnings("ignore", category=DeprecationWarning)
random.seed("cee")

Could not import SolaraViz. If you need it, install with 'pip install --pre mesa[viz]'


In [5]:
class CommunityAgent(Agent):
    def __init__(self, unique_id, model, asset_probability=0.5):
        # Initialize the agent
        self.unique_id = unique_id
        self.model = model
        self.pos = None  # Initialize position attribute

            # Attributes
        self.money = model.random.randint(10, 100)
        self.assets = (
            model.random.randint(1, 10) if model.random.random() < asset_probability else 0
        )
        self.social_ties = {}  # Relationships with other agents
        self.trade_state = None  # 'buy', 'sell', or None
        self.trade_quantity = 0  # Quantity to buy or sell
        self.value_preference = self.calculate_value_preference()  # Dynamically calculated

    def calculate_value_preference(self):
        """Calculate value preference based on assets and social ties."""
        def compute_social_influence():
            """Helper to compute influence from social ties with owners."""
            return sum(
                max(0, tie_strength) * 0.25
                for neighbor_id, tie_strength in self.social_ties.items()
                if self.is_neighbor_owner(neighbor_id)
            )

        def compute_asset_preference():
            """Helper to compute preference based on owned assets with diminishing returns."""
            scaling_factor = 2  # Increment per asset owned
            return sum(scaling_factor / (i + 1) for i in range(self.assets))

        if self.assets == 0:
            # Non-owners base their preference solely on social influence
            return compute_social_influence() if self.social_ties else 0

        # Owners combine asset preference and social influence
        return compute_asset_preference() + compute_social_influence()

    def is_neighbor_owner(self, neighbor_id):
        """Check if a neighbor owns assets."""
        neighbor = next((a for a in self.model.schedule.agents if a.unique_id == neighbor_id), None)
        return neighbor and neighbor.assets > 0

    def interact(self):
        """Interact with a neighbor and adjust social ties."""
        # Choose a random neighbor to interact with
        neighbors = list(self.model.grid.get_neighbors(self.unique_id, include_center=False))

        if not neighbors:
            return  # No neighbors to interact with

        # Compute weights based on social ties
        weights = [
            max(0, 1 + self.social_ties.get(neighbor_id, 0)) for neighbor_id in neighbors
        ]

        # Normalize weights to sum to 1
        total_weight = sum(weights)
        weights = [w / total_weight for w in weights] if total_weight > 0 else [1 / len(neighbors)] * len(neighbors)

        # Select a neighbor based on weights
        partner_id = self.model.random.choices(neighbors, weights=weights, k=1)[0]

        # Determine interaction outcome
        interaction_outcome = self.model.random.choice(['positive', 'negative'])

        # Update social tie strength
        tie_strength = self.social_ties.get(partner_id, 0)
        self.social_ties[partner_id] = tie_strength + 1 if interaction_outcome == 'positive' else tie_strength - 1

    def decide_trade(self):
        """Decide whether to buy or sell, and the quantity."""
        # Update value preference dynamically
        self.value_preference = self.calculate_value_preference()

        # Decide to buy or sell based on current price and value preference
        price = self.model.market_maker['price']

        if self.money > price and self.model.random.random() > 0.5:
            self.trade_state = 'buy'
            max_affordable = int(self.money / price)
            self.trade_quantity = min(max_affordable, 10)  # Max of 10 assets per trade
        elif self.assets > 0 and self.model.random.random() > 0.5:
            self.trade_state = 'sell'
            self.trade_quantity = min(self.assets, 10)  # Max of 10 assets per trade
        else:
            self.trade_state = None
            self.trade_quantity = 0

    def trade(self):
        """Perform trade with the market maker."""
        price = self.model.market_maker['price']
        if self.trade_state == 'buy' and self.trade_quantity > 0:
            total_cost = self.trade_quantity * price
            if self.money >= total_cost and self.model.market_maker['inventory'] >= self.trade_quantity:
                self.money -= total_cost
                self.assets += self.trade_quantity
                self.model.market_maker['inventory'] -= self.trade_quantity

        elif self.trade_state == 'sell' and self.trade_quantity > 0:
            total_revenue = self.trade_quantity * price
            if self.assets >= self.trade_quantity:
                self.money += total_revenue
                self.assets -= self.trade_quantity
                self.model.market_maker['inventory'] += self.trade_quantity

        # Update value preference after trading
        self.value_preference = self.calculate_value_preference()

    def step(self):
        """Run one step of the agent's behavior."""
        self.interact()
        self.decide_trade()
        self.trade()

In [6]:
class CommunityModel(Model):
    def __init__(self, num_agents):
        super().__init__()  # Initialize the parent Model class
        self.random = random.Random()  # Initialize the model's random generator
        self.num_agents = num_agents
        self.schedule = RandomActivation(self)

        # Define networks
        self.interaction_network = nx.newman_watts_strogatz_graph(n=num_agents, k=3, p=0.1)
        self.exchange_network = nx.star_graph(n=num_agents)  # Star network with market maker at the center
        self.grid = NetworkGrid(self.interaction_network)

        # Market maker
        self.market_maker = {'price': 10, 'inventory': 10000}

        # Historical data tracker
        self.history = []

        # Add agents
        for i in range(num_agents):
            agent = CommunityAgent(i, self, asset_probability=0.1)
            self.schedule.add(agent)
            if not self.grid.is_cell_empty(i):
                raise Exception("Grid cell is not empty")
            self.grid.place_agent(agent, i)

    def update_price(self):
        """Adjust price based on supply and demand."""
        total_buy = sum(agent.trade_quantity for agent in self.schedule.agents if agent.trade_state == 'buy')
        total_sell = sum(agent.trade_quantity for agent in self.schedule.agents if agent.trade_state == 'sell')

        # Adjust price based on the net demand
        self.market_maker['price'] += 0.1 * (total_buy - total_sell)
        self.market_maker['price'] = max(self.market_maker['price'], 1)  # Ensure price stays positive

    def step(self):
        """Advance the model by one step."""
        # Activate agents
        self.schedule.step()
        # Update the market price
        self.update_price()

        # Compute averages for owners and non-owners
        owners = []
        non_owners = []
        total_assets = 0

        for agent in self.schedule.agents:
            total_assets += agent.assets  # Sum total assets
            if agent.assets > 0:
                owners.append(agent.value_preference)
            else:
                # Compute non-owner preference based on social ties with owners
                social_influence = sum(
                    max(0, tie_strength) * 0.1
                    for neighbor, tie_strength in agent.social_ties.items()
                    if neighbor.assets > 0
                )
                non_owners.append(social_influence)

        avg_owners = sum(owners) / len(owners) if owners else 0
        avg_non_owners = sum(non_owners) / len(non_owners) if non_owners else 0
        avg_assets = total_assets / len(self.schedule.agents)  # Average assets per agent

        # Record history
        self.history.append({
            "step": len(self.history),
            "average_value_preference_owners": avg_owners,
            "average_value_preference_non_owners": avg_non_owners,
            "average_assets": avg_assets,
        })

    def get_visualization_data(self):
        """Prepare data for Altair visualization."""
        # Nodes data
        nodes = []
        for agent in self.schedule.agents:
            nodes.append({
                "id": agent.unique_id,
                "assets": agent.assets,
                "value_preference": agent.value_preference,
                "color": "orange" if agent.assets > 0 else "gray",  # Blue if owning assets
                "size": 100 + agent.assets * 50,  # Node size scales with assets
            })
        # Check nodes data
        print("Nodes Data:", nodes)
        nodes_df = pl.DataFrame(nodes)

        # Edges data
        edges = []
        for agent in self.schedule.agents:
            for neighbor, tie_strength in agent.social_ties.items():
                edges.append({
                    "source": agent.unique_id,
                    "target": neighbor.unique_id,
                    "tie_strength": tie_strength,
                    "color": "blue" if tie_strength > 0 else ("red" if tie_strength < 0 else "gray"),
                    "width": abs(tie_strength) * 2,  # Edge width scales with tie strength
                })
        # Check edges data
        print("Edges Data:", edges)
        edges_df = pl.DataFrame(edges)

        return nodes_df, edges_df

    def visualize_network(self):
        """Visualize the model using Altair with a force-directed layout."""
        nodes_df, edges_df = self.get_visualization_data()

        # Convert Polars DataFrames to Pandas for Altair compatibility
        nodes_df = nodes_df.to_pandas()
        edges_df = edges_df.to_pandas()

        # Create a NetworkX graph from the edges data
        G = nx.Graph()
        for _, edge in edges_df.iterrows():
            G.add_edge(edge["source"], edge["target"], weight=edge["tie_strength"])

        # Compute a force-directed layout for the graph
        pos = nx.spring_layout(G)

        # Add layout positions to the nodes DataFrame
        nodes_df["x"] = nodes_df["id"].apply(lambda node: pos[node][0])
        nodes_df["y"] = nodes_df["id"].apply(lambda node: pos[node][1])

        # Add layout positions to the edges DataFrame
        edges_df["x"] = edges_df["source"].apply(lambda node: pos[node][0])
        edges_df["y"] = edges_df["source"].apply(lambda node: pos[node][1])
        edges_df["x2"] = edges_df["target"].apply(lambda node: pos[node][0])
        edges_df["y2"] = edges_df["target"].apply(lambda node: pos[node][1])

        # Define edge layer
        edges_chart = alt.Chart(edges_df).mark_rule().encode(
            x="x:Q",
            x2="x2:Q",
            y="y:Q",
            y2="y2:Q",
            strokeWidth=alt.Size("width:Q", scale=alt.Scale(range=[1, 5]), legend=None),
            color=alt.Color("color:N", scale=None, legend=None),
        )

        # Define node layer
        nodes_chart = alt.Chart(nodes_df).mark_circle().encode(
            x="x:Q",
            y="y:Q",
            size=alt.Size("size:Q", scale=alt.Scale(range=[100, 1000]), legend=None),
            color=alt.Color("color:N", scale=None, legend=None),
            tooltip=["id", "assets", "value_preference"],
        )

        # Combine node and edge layers
        chart = alt.layer(edges_chart, nodes_chart).configure_view(strokeWidth=0).configure_axis(grid=False)
        return chart

    def visualize_time_series(self):
        """Visualize average value preferences for owners and non-owners on the same chart, with a legend."""
        # Convert history to a Polars DataFrame
        history_df = pl.DataFrame(self.history)

        # Reshape the data for a single chart with a legend
        reshaped_df = history_df.select([
            pl.col("step").alias("Step"),
            pl.col("average_value_preference_owners").alias("Owners"),
            pl.col("average_value_preference_non_owners").alias("Non-Owners"),
        ]).melt(
            id_vars=["Step"],
            value_vars=["Owners", "Non-Owners"],
            variable_name="Group",
            value_name="Preference"
        )

        # Create the line chart for value preferences
        preference_chart = alt.Chart(reshaped_df).mark_line().encode(
            x=alt.X("Step:Q", title="Time Step"),
            y=alt.Y("Preference:Q", title="Average Value Preference"),
            color=alt.Color(
                "Group:N",
                scale=alt.Scale(domain=["Owners", "Non-Owners"], range=["blue", "green"]),
                title="Group"
            ),
        ).properties(width=600, height=300)

        # Create the secondary line chart for average assets
        assets_chart = alt.Chart(history_df).mark_point(filled=True).encode(
            x=alt.X("step:Q", title="Time Step"),
            y=alt.Y("average_assets:Q", title="Average # of Assets Held"),
            color=alt.value("black"),
        )

        # Combine the charts
        combined_chart = alt.layer(preference_chart, assets_chart).resolve_scale(
            y="independent"  # Independent scales for dual axes
        ).properties(title="Average Value Preferences (Owners vs Non-Owners) and Average Assets Over Time")

        return combined_chart


In [7]:
# Initialize the model with 16 agents
model = CommunityModel(num_agents=32)

# Run for 10 steps
for step in range(1000):
    model.step()
    if step % 50 == 0:
        print(f"Step {step}: Price = {model.market_maker['price']:.2f}")


Step 0: Price = 14.30
Step 50: Price = 12.90
Step 100: Price = 25.00
Step 150: Price = 27.70
Step 200: Price = 38.90
Step 250: Price = 41.60
Step 300: Price = 43.50
Step 350: Price = 41.40
Step 400: Price = 58.50
Step 450: Price = 59.50
Step 500: Price = 54.20
Step 550: Price = 65.20
Step 600: Price = 69.60
Step 650: Price = 64.00
Step 700: Price = 73.90
Step 750: Price = 78.90
Step 800: Price = 77.10
Step 850: Price = 92.80
Step 900: Price = 81.10
Step 950: Price = 96.90


In [8]:
model.visualize_network()

Nodes Data: [{'id': 17, 'assets': 20, 'value_preference': 7.195479314287364, 'color': 'orange', 'size': 1100}, {'id': 2, 'assets': 2, 'value_preference': 3.0, 'color': 'orange', 'size': 200}, {'id': 7, 'assets': 0, 'value_preference': 0, 'color': 'gray', 'size': 100}, {'id': 21, 'assets': 0, 'value_preference': 0, 'color': 'gray', 'size': 100}, {'id': 6, 'assets': 49, 'value_preference': 8.958410676658847, 'color': 'orange', 'size': 2550}, {'id': 20, 'assets': 9, 'value_preference': 5.657936507936507, 'color': 'orange', 'size': 550}, {'id': 29, 'assets': 0, 'value_preference': 0, 'color': 'gray', 'size': 100}, {'id': 8, 'assets': 9, 'value_preference': 5.657936507936507, 'color': 'orange', 'size': 550}, {'id': 25, 'assets': 36, 'value_preference': 8.349118393589277, 'color': 'orange', 'size': 1900}, {'id': 18, 'assets': 66, 'value_preference': 9.548854068483797, 'color': 'orange', 'size': 3400}, {'id': 12, 'assets': 46, 'value_preference': 8.833374491972208, 'color': 'orange', 'size': 

alt.LayerChart(...)

In [9]:
model.visualize_time_series()

alt.LayerChart(...)